In [86]:
import numpy as np
from helpers import *
from costs import *
from gradient_descent import *
from stochastic_gradient_descent import *


In [87]:
y, x, ids = load_csv_data(data_path="datas/train.csv", sub_sample=False)
x.shape

(250000, 30)

In [88]:
w = np.random.rand(x.shape[1])
compute_loss(y, x, w)

11213574.010750543

In [89]:
gradient = compute_gradient(y, x, w)
gradient.shape

(30,)

In [102]:
w_init = np.random.rand(x.shape[1])
w, loss = least_squares(y, x, w_init, max_iters=2000, gamma=0.0000003)

Gradient Descent(0/1999): loss=7676948.342339236, w0=0.22215848824703435, w1=0.9960461611852883
Gradient Descent(1/1999): loss=4028828.03506174, w0=0.2738128068200162, w1=0.9571032684021228
Gradient Descent(2/1999): loss=2126111.4790488547, w0=0.20038360679531597, w1=0.9824314188387898
Gradient Descent(3/1999): loss=1131603.5765226516, w0=0.22115656037586062, w1=0.9615333420596536
Gradient Descent(4/1999): loss=610187.838159187, w0=0.1776273871326181, w1=0.9740070651345656
Gradient Descent(5/1999): loss=335586.03719589906, w0=0.183522458726586, w1=0.9625082066584035
Gradient Descent(6/1999): loss=190026.21821793218, w0=0.15652029110371268, w1=0.9683443015515092
Gradient Descent(7/1999): loss=112139.56520895318, w0=0.15558489938428294, w1=0.9617552585404278
Gradient Descent(8/1999): loss=69895.6563742921, w0=0.13794667466174432, w1=0.9641774278798004
Gradient Descent(9/1999): loss=46538.56459340381, w0=0.13416390401677983, w1=0.9601659187008593
Gradient Descent(10/1999): loss=33274.9042

Gradient Descent(85/1999): loss=2599.958551146894, w0=0.02925577922996468, w1=0.905847632440395
Gradient Descent(86/1999): loss=2577.9380954637395, w0=0.02926361926339081, w1=0.9053523885630579
Gradient Descent(87/1999): loss=2556.322611816459, w0=0.02927344756325896, w1=0.9048593311928239
Gradient Descent(88/1999): loss=2535.099119301575, w0=0.02928508570047317, w1=0.9043684036437948
Gradient Descent(89/1999): loss=2514.255406764578, w0=0.029298366909924708, w1=0.903879551329863
Gradient Descent(90/1999): loss=2493.779972301171, w0=0.029313135402445463, w1=0.9033927216815538
Gradient Descent(91/1999): loss=2473.6619678742545, w0=0.029329245716420056, w1=0.9029078640657843
Gradient Descent(92/1999): loss=2453.8911486056036, w0=0.029346562105365175, w1=0.9024249297091593
Gradient Descent(93/1999): loss=2434.457826339514, w0=0.029364957960504506, w1=0.9019438716241317
Gradient Descent(94/1999): loss=2415.3528271107743, w0=0.02938431526562532, w1=0.9014646445382932
Gradient Descent(95/199

Gradient Descent(168/1999): loss=1571.4124573192837, w0=0.030619423239118315, w1=0.8692767100368203
Gradient Descent(169/1999): loss=1564.9646215406747, w0=0.030625302238984357, w1=0.8688722973674712
Gradient Descent(170/1999): loss=1558.6010332930546, w0=0.030630949493507524, w1=0.8684684934314174
Gradient Descent(171/1999): loss=1552.320389245945, w0=0.03063636905034935, w1=0.868065293455239
Gradient Descent(172/1999): loss=1546.1214068214922, w0=0.030641564896843238, w1=0.8676626927557103
Gradient Descent(173/1999): loss=1540.0028238371071, w0=0.030646540958714576, w1=0.8672606867369207
Gradient Descent(174/1999): loss=1533.9633981564239, w0=0.03065130109902226, w1=0.8668592708875056
Gradient Descent(175/1999): loss=1528.0019073481958, w0=0.030655849117304346, w1=0.8664584407779836
Gradient Descent(176/1999): loss=1522.1171483528105, w0=0.03066018874891203, w1=0.8660581920581962
Gradient Descent(177/1999): loss=1516.3079371561257, w0=0.030664323664516935, w1=0.8656585204548449
Gradi

Gradient Descent(251/1999): loss=1233.1735619252588, w0=0.030583541775624806, w1=0.8374541267651101
Gradient Descent(252/1999): loss=1230.6966227034884, w0=0.030578874450771076, w1=0.8370890613091133
Gradient Descent(253/1999): loss=1228.243261433823, w0=0.030574140717756323, w1=0.836724371443737
Gradient Descent(254/1999): loss=1225.813126279206, w0=0.030569341325569883, w1=0.8363600554366287
Gradient Descent(255/1999): loss=1223.4058708822095, w0=0.030564477010442456, w1=0.8359961115699808
Gradient Descent(256/1999): loss=1221.0211542793759, w0=0.030559548496176696, w1=0.8356325381403465
Gradient Descent(257/1999): loss=1218.658640816895, w0=0.030554556494468164, w1=0.835269333458462
Gradient Descent(258/1999): loss=1216.318000067606, w0=0.03054950170521695, w1=0.8349064958490703
Gradient Descent(259/1999): loss=1213.9989067492963, w0=0.030544384816830143, w1=0.834544023650751
Gradient Descent(260/1999): loss=1211.7010406442805, w0=0.030539206506515423, w1=0.834181915215753
Gradient 

Gradient Descent(334/1999): loss=1083.508879194324, w0=0.030022377898500336, w1=0.8082952297596697
Gradient Descent(335/1999): loss=1082.169852908627, w0=0.03001398030568104, w1=0.8079565279098581
Gradient Descent(336/1999): loss=1080.8379379473326, w0=0.03000555343304346, w1=0.80761809555687
Gradient Descent(337/1999): loss=1079.5130382551793, w0=0.029997097551645428, w1=0.8072799317800426
Gradient Descent(338/1999): loss=1078.1950592598475, w0=0.029988612930088667, w1=0.8069420356652703
Gradient Descent(339/1999): loss=1076.8839078488008, w0=0.029980099834550224, w1=0.8066044063049519
Gradient Descent(340/1999): loss=1075.579492346492, w0=0.029971558528813187, w1=0.8062670427979378
Gradient Descent(341/1999): loss=1074.2817224919233, w0=0.029962989274296747, w1=0.805929944249478
Gradient Descent(342/1999): loss=1072.9905094165604, w0=0.029954392330085566, w1=0.8055931097711699
Gradient Descent(343/1999): loss=1071.705765622585, w0=0.02994576795295854, w1=0.8052565384809081
Gradient D

...

Gradient Descent(1667/1999): loss=373.95692611275155, w0=0.017086261184416092, w1=0.48738347963443385
Gradient Descent(1668/1999): loss=373.70690396851774, w0=0.01707835981817605, w1=0.48721068324485306
Gradient Descent(1669/1999): loss=373.457107211129, w0=0.017070461292694974, w1=0.4870379635985799
Gradient Descent(1670/1999): loss=373.20753560085865, w0=0.017062565607016498, w1=0.48686532065424404
Gradient Descent(1671/1999): loss=372.95818889826563, w0=0.01705467276018448, w1=0.48669275437050163
Gradient Descent(1672/1999): loss=372.70906686419306, w0=0.017046782751243005, w1=0.4865202647060355
Gradient Descent(1673/1999): loss=372.4601692597693, w0=0.017038895579236386, w1=0.48634785161955496
Gradient Descent(1674/1999): loss=372.2114958464061, w0=0.01703101124320917, w1=0.4861755150697959
Gradient Descent(1675/1999): loss=371.9630463857997, w0=0.017023129742206124, w1=0.4860032550155206
Gradient Descent(1676/1999): loss=371.714820639929, w0=0.01701525107527226, w1=0.4858310714155

Gradient Descent(1749/1999): loss=354.1828788025799, w0=0.016447699025595827, w1=0.4734654224159578
Gradient Descent(1750/1999): loss=353.95056687580666, w0=0.016440027446288877, w1=0.4732987839066029
Gradient Descent(1751/1999): loss=353.71846159373644, w0=0.016432358630105854, w1=0.47313221883473694
Gradient Descent(1752/1999): loss=353.4865627388499, w0=0.016424692576110168, w1=0.4729657271611047
Gradient Descent(1753/1999): loss=353.2548700938842, w0=0.016417029283365466, w1=0.4727993088464759
Gradient Descent(1754/1999): loss=353.0233834418326, w0=0.016409368750935655, w1=0.4726329638516454
Gradient Descent(1755/1999): loss=352.7921025659443, w0=0.01640171097788489, w1=0.4724666921374328
Gradient Descent(1756/1999): loss=352.5610272497246, w0=0.016394055963277577, w1=0.4723004936646829
Gradient Descent(1757/1999): loss=352.33015727693345, w0=0.016386403706178384, w1=0.47213436839426526
Gradient Descent(1758/1999): loss=352.09949243158604, w0=0.016378754205652216, w1=0.471968316287

Gradient Descent(1831/1999): loss=335.80074430371985, w0=0.01582772335683581, w1=0.4600415173420698
Gradient Descent(1832/1999): loss=335.5846770200634, w0=0.015820275265835806, w1=0.45988077252271997
Gradient Descent(1833/1999): loss=335.3687993682467, w0=0.015812829862009044, w1=0.459720098003364
Gradient Descent(1834/1999): loss=335.15311115073536, w0=0.01580538714444008, w1=0.4595594937467372
Gradient Descent(1835/1999): loss=334.9376121702261, w0=0.015797947112213714, w1=0.45939895971559846
Gradient Descent(1836/1999): loss=334.7223022296461, w0=0.015790509764415026, w1=0.45923849587273025
Gradient Descent(1837/1999): loss=334.507181132153, w0=0.01578307510012935, w1=0.4590781021809384
Gradient Descent(1838/1999): loss=334.2922486811347, w0=0.015775643118442286, w1=0.45891777860305244
Gradient Descent(1839/1999): loss=334.07750468020885, w0=0.0157682138184397, w1=0.45875752510192525
Gradient Descent(1840/1999): loss=333.8629489332224, w0=0.015760787199207718, w1=0.4585973416404332

Gradient Descent(1913/1999): loss=318.6958968835209, w0=0.015225823386001269, w1=0.44709064900388706
Gradient Descent(1914/1999): loss=318.4947433525431, w0=0.015218592626365676, w1=0.44693554708411015
Gradient Descent(1915/1999): loss=318.293763997361, w0=0.015211364479713978, w1=0.4467805124853445
Gradient Descent(1916/1999): loss=318.092958638439, w0=0.015204138945152443, w1=0.44662554517220115
Gradient Descent(1917/1999): loss=317.8923270964499, w0=0.015196916021787616, w1=0.4464706451093133
Gradient Descent(1918/1999): loss=317.691869192274, w0=0.01518969570872629, w1=0.4463158122613362
Gradient Descent(1919/1999): loss=317.491584747, w0=0.015182478005075552, w1=0.4461610465929475
Gradient Descent(1920/1999): loss=317.2914735819229, w0=0.015175262909942724, w1=0.44600634806884676
Gradient Descent(1921/1999): loss=317.09153551854547, w0=0.015168050422435417, w1=0.4458517166537559
Gradient Descent(1922/1999): loss=316.89177037857723, w0=0.0151608405416615, w1=0.44569715231241874
Gra

Gradient Descent(1995/1999): loss=302.76420247640567, w0=0.014641500133110818, w1=0.43459276900735616
Gradient Descent(1996/1999): loss=302.5767531046238, w0=0.014634480693695143, w1=0.43444307182208086
Gradient Descent(1997/1999): loss=302.3894638552242, w0=0.014627463794863196, w1=0.43429343912825635
Gradient Descent(1998/1999): loss=302.2023345648946, w0=0.01462044943574311, w1=0.43414387089226236
Gradient Descent(1999/1999): loss=302.0153650705096, w0=0.014613437615463298, w1=0.43399436708049954


In [97]:
# w, loss = least_squares_SGD(y, x, w, 10, max_iters=100, gamma=0.0000001)

In [103]:
def get_accuracy(x, y, w):
    pred_y = predict_labels(w, x)
    correct_count = 0
    for index, yi in enumerate(y):
        pred_yi = pred_y[index]
        if pred_yi == yi:
            correct_count += 1
    return correct_count/len(y)
        
get_accuracy(x, y, w)

0.431368

In [104]:
y_test, x_test, ids_test = load_csv_data(data_path="datas/test.csv", sub_sample=False)
pred_y = predict_labels(w, x_test)


In [105]:
def generate_submission_csv(ids, y):
    submission = np.zeros((y.shape[0], 2))
    submission[:,1] = y
    submission[:,0] = ids
    np.savetxt("datas/submission.csv", submission, fmt='%i', delimiter=",")
    
    def line_prepender(filename, line):
        with open(filename, 'r+') as f:
            content = f.read()
            f.seek(0, 0)
            f.write(line.rstrip('\r\n') + '\n' + content)
    line_prepender("datas/submission.csv", "Id,Prediction")
    
    print('Done !')


In [106]:
generate_submission_csv(ids_test, pred_y)

Done !
